In [1]:
import polars as pl
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from time import perf_counter


In [2]:
orig_x, orig_y = make_classification(n_samples = 100_000, n_features = 10, n_informative = 5, n_redundant = 5)
# This is a Polars dataframe. This is dsds package's favored dataframe. dsds relies on Polars heavily.
# You must turn other dataframe formats into Polars for dsds to work.
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y)) 
# Turn it into Pandas.
df_pd = df.to_pandas()
target = "target"
features = df.columns
features.remove(target)

In [3]:
df.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.917932,3.395753,-3.033801,-3.373439,-3.231625,-5.664714,-5.296789,-1.977015,-0.617106,-2.291159
0,-0.615102,2.570032,-2.660985,-2.744959,-2.174712,-3.225323,-2.264492,-1.696224,0.969224,-1.841991
0,-0.090845,1.387114,-2.078553,-1.84527,-2.093092,-3.08458,-1.534529,-2.883363,1.714375,-1.383043
1,0.420971,-2.242455,5.224822,1.462509,2.609278,3.792593,2.093241,-1.697945,-0.881956,1.475591
1,-2.203745,2.020452,0.608961,-2.083744,0.390127,1.502105,0.511588,-0.176106,0.091584,-0.677401


## Comparisons

This notebook compares results and performance between the dsds package, sklearn and some other packages for feature selection and some other transformations common in the data science pipeline.

### Methods Compared:
1. Scaling and Imputation
2. Fscore
3. Mutual Information Score
4. MRMR feature selection strategies
5. Power Transform

You may restart the kernel after each section. But remember to rerun the cells above. If you are concerned about memory usage when running this notebook, go to the end and run the gc cell.

# Scaling and Imputation

In [4]:
import dsds.transform as t

In [5]:
features = df.columns
features.remove("target")

In [6]:
scaled = t.scale(df, cols=features, strategy="standard")
scaled.head()

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.880607,1.333267,-0.66621,-1.110481,-1.967191,-1.80741,-2.115968,-1.279214,-0.729238,-0.845485
0,-0.071313,0.952163,-0.534734,-0.818885,-1.3226,-0.928822,-0.898976,-1.097703,0.303606,-0.550253
0,0.254218,0.406198,-0.329335,-0.401456,-1.272822,-0.878131,-0.60601,-1.865102,0.788766,-0.248592
1,0.572025,-1.268996,2.246255,1.133256,1.595061,1.598799,0.849971,-1.098815,-0.901679,1.630351
1,-1.057764,0.698509,0.618437,-0.512101,0.241644,0.773841,0.215186,-0.115057,-0.267816,0.215217


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# The difference in result is caused by using ddof = 1 for sample variance in dsds
# and using ddof = 0 in sklearn.

# Long and convoluted code just to do some scaling...
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
# scaled2[:5, :] # scaled2 is a numpy matrix
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]
scaled2.head()

,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,target
0,0.880611,1.333273,-0.666214,-1.110487,-1.967200,-1.807419,-2.115978,-1.279220,-0.729241,-0.845489,0
1,-0.071314,0.952168,-0.534737,-0.818889,-1.322607,-0.928827,-0.898980,-1.097709,0.303607,-0.550255,0
2,0.254220,0.406200,-0.329336,-0.401458,-1.272828,-0.878136,-0.606013,-1.865111,0.788770,-0.248594,0
3,0.572028,-1.269002,2.246267,1.133261,1.595069,1.598807,0.849976,-1.098821,-0.901684,1.630359,1
4,-1.057769,0.698513,0.618440,-0.512103,0.241645,0.773845,0.215187,-0.115058,-0.267818,0.215218,1


In [9]:
%%timeit
scaled = t.scale(df, cols=features, strategy="standard")

2.73 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
std = StandardScaler()
scaled2 = std.fit_transform(df_pd[features], df_pd[target])
scaled2 = pd.DataFrame(scaled2, columns=features)
scaled2[target] = df_pd[target]


13.7 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
t.impute(df, cols=features, strategy="median").head() 

target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.917932,3.395753,-3.033801,-3.373439,-3.231625,-5.664714,-5.296789,-1.977015,-0.617106,-2.291159
0,-0.615102,2.570032,-2.660985,-2.744959,-2.174712,-3.225323,-2.264492,-1.696224,0.969224,-1.841991
0,-0.090845,1.387114,-2.078553,-1.84527,-2.093092,-3.08458,-1.534529,-2.883363,1.714375,-1.383043
1,0.420971,-2.242455,5.224822,1.462509,2.609278,3.792593,2.093241,-1.697945,-0.881956,1.475591
1,-2.203745,2.020452,0.608961,-2.083744,0.390127,1.502105,0.511588,-0.176106,0.091584,-0.677401


In [12]:
from sklearn.impute import SimpleImputer 

imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)
imputed.head()

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0.0,0.917932,3.395753,-3.033801,-3.373439,-3.231625,-5.664714,-5.296789,-1.977015,-0.617106,-2.291159
1,0.0,-0.615102,2.570032,-2.660985,-2.744959,-2.174712,-3.225323,-2.264492,-1.696224,0.969224,-1.841991
2,0.0,-0.090845,1.387114,-2.078553,-1.845270,-2.093092,-3.084580,-1.534529,-2.883363,1.714375,-1.383043
3,1.0,0.420971,-2.242455,5.224822,1.462509,2.609278,3.792593,2.093241,-1.697945,-0.881956,1.475591
4,1.0,-2.203745,2.020452,0.608961,-2.083744,0.390127,1.502105,0.511588,-0.176106,0.091584,-0.677401


In [13]:
%%timeit
t.impute(df, cols=features, strategy="median")

1.27 ms ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
imputer = SimpleImputer(strategy = "median")
imputed = pd.DataFrame(imputer.fit_transform(df_pd, df_pd[target]), columns=df.columns)

114 ms ± 388 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Fscore

In [15]:
import dsds.fs as fs # fs = feature_selection
from sklearn.feature_selection import mutual_info_classif, f_classif, f_regression

In [16]:
fs._f_score(df, target=target, num_list = features)

array([1.02017433e+04, 1.24032529e+04, 4.39965363e+03, 1.41390263e+03,
       5.75678286e+04, 2.57446403e+04, 5.20532620e+04, 3.44951164e-01,
       1.15913839e+04, 3.24985962e+00])

In [17]:
# The more core, the bigger the difference. Data here is not big enough to show the difference
start = perf_counter()
res = fs.f_classif(df, target=target)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.00s in computing Fscore.


feature,f_value,p_value
str,f64,f64
"""column_0""",10201.74326,0.0
"""column_1""",12403.252938,0.0
"""column_2""",4399.653633,0.0
"""column_3""",1413.902625,2.8493e-307
"""column_4""",57567.828579,0.0
"""column_5""",25744.640337,0.0
"""column_6""",52053.261985,0.0
"""column_7""",0.344951,0.556986
"""column_8""",11591.383874,0.0


In [18]:
start = perf_counter()
f, pv = f_classif(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing Fscore.")
res.head(10)

Spent 0.02s in computing Fscore.


,feature,f_value,p_value
0,column_0,10201.743260,0.000000e+00
1,column_1,12403.252938,0.000000e+00
2,column_2,4399.653633,0.000000e+00
3,column_3,1413.902625,2.849265e-307
4,column_4,57567.828579,0.000000e+00
5,column_5,25744.640337,0.000000e+00
6,column_6,52053.261985,0.000000e+00
7,column_7,0.344951,5.569865e-01
8,column_8,11591.383874,0.000000e+00
9,column_9,3.249860,7.143258e-02


In [19]:
start = perf_counter()
f, pv = f_regression(df_pd[features], df_pd[target])
res = pd.DataFrame({"feature":features, "f_value":f, "p_value":pv})
end = perf_counter()
print(f"Spent {end - start: .2f}s in computing Fscore.")
res.head(10)

Spent  0.01s in computing Fscore.


,feature,f_value,p_value
0,column_0,10201.743260,0.000000e+00
1,column_1,12403.252938,0.000000e+00
2,column_2,4399.653633,0.000000e+00
3,column_3,1413.902625,2.849265e-307
4,column_4,57567.828579,0.000000e+00
5,column_5,25744.640337,0.000000e+00
6,column_6,52053.261985,0.000000e+00
7,column_7,0.344951,5.569865e-01
8,column_8,11591.383874,0.000000e+00
9,column_9,3.249860,7.143258e-02


# Mutual Information Score

In [20]:
# Vastly faster than sklearn. Finished in 0.7s in this run
fs.mutual_info(df, target=target, conti_cols=features).sort(by="estimated_mi", descending=True).limit(10)

Mutual Info: 100%|██████████| 10/10 [00:00<00:00, 13.22it/s]


feature,estimated_mi
str,f64
"""column_6""",0.262854
"""column_4""",0.235594
"""column_5""",0.134383
"""column_1""",0.060619
"""column_8""",0.057225
"""column_0""",0.054333
"""column_2""",0.025232
"""column_7""",0.00989
"""column_3""",0.007628


In [21]:
def estimate_mi_sklearn(df:pd.DataFrame, cols:list[str], target:str, k=3, random_state:int=42):
    mi_estimates = mutual_info_classif(df[cols], df[target]
                        , n_neighbors=k, random_state=random_state, discrete_features=False)

    return pl.from_records([cols, mi_estimates], schema=["feature", "estimated_mi"]).sort("estimated_mi", descending=True)

In [22]:
# The reason sklearn's impl is slow is that it did not turn on multithreading for KDtrees.
# Sklearn also did not provide an option to turn it on, despite the fact that sklearn's KDtrees
# does have this functionality. Finished in 4.4s in this run
estimate_mi_sklearn(df_pd, cols=features, target=target).limit(10)

feature,estimated_mi
str,f64
"""column_6""",0.262857
"""column_4""",0.235594
"""column_5""",0.134383
"""column_1""",0.060619
"""column_8""",0.057227
"""column_0""",0.054333
"""column_2""",0.025232
"""column_7""",0.009889
"""column_3""",0.007626


# MRMR Feature selection Strategy

In [23]:
from mrmr import mrmr_classif # This is currently the most starred MRMR Python package on github

In [24]:
# Need to wrap it so that we get apples to apples comparison
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    features = list(df.columns)
    features.remove(target)
    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [25]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 10/10 [00:05<00:00,  1.70it/s]

Spent 8.70s to compute mrmr.


['column_4',
 'column_8',
 'column_6',
 'column_5',
 'column_0',
 'column_1',
 'column_2',
 'column_3',
 'column_9',
 'column_7']

In [26]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=False)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 1999.48it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.02s in computing.


['column_4',
 'column_8',
 'column_6',
 'column_5',
 'column_0',
 'column_1',
 'column_2',
 'column_3',
 'column_9',
 'column_7']

In [27]:
start = perf_counter()
res = fs.mrmr(df, target="target", k = 50, low_memory=True)
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res

Running fscore to determine feature relevance...
Found 10 total features to select from. Proceeding to select top 10 features.


MRMR, fscore: 100%|██████████| 10/10 [00:00<00:00, 526.17it/s]

Output is sorted in order of selection (max relevance min redundancy).
Spent 0.03s in computing.


['column_4',
 'column_8',
 'column_6',
 'column_5',
 'column_0',
 'column_1',
 'column_2',
 'column_3',
 'column_9',
 'column_7']

# Power Transform

In [28]:
# Eager transform.
start = perf_counter()
res_eager = t.power_transform(df, cols=features, strategy="yeo_johnson")
end = perf_counter()
print(f"Spent {end - start:.2f}s in computing.")
res_eager.head() 

Spent 0.23s in computing.


target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.864847,3.435183,-3.153299,-3.436937,-2.897577,-4.785028,-4.0769,-2.049778,-0.605948,-2.308299
0,-0.642348,2.595,-2.757164,-2.790259,-1.998734,-2.855113,-1.936529,-1.752109,0.995407,-1.853833
0,-0.091515,1.395827,-2.142008,-1.868485,-1.927902,-2.739654,-1.362059,-3.020756,1.786407,-1.390254
1,0.408198,-2.222789,4.955917,1.447182,2.875639,4.35392,2.440398,-1.753928,-0.860575,1.467571
1,-2.47669,2.037121,0.601976,-2.112301,0.398657,1.617501,0.537919,-0.176931,0.091872,-0.679397


In [29]:
from sklearn.preprocessing import power_transform

In [30]:
# Sklearn with Pandas

start = perf_counter()
transformed = power_transform(df_pd[features], method = "yeo-johnson", standardize=False)
end = perf_counter()
df_pd[features] = transformed
print(f"Spent {end - start:.2f}s in computing.")
df_pd.head()


Spent 0.69s in computing.


,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9
0,0,0.864847,3.435183,-3.153299,-3.436937,-2.897577,-4.785028,-4.076900,-2.049778,-0.605948,-2.308299
1,0,-0.642348,2.595000,-2.757164,-2.790259,-1.998734,-2.855113,-1.936529,-1.752109,0.995407,-1.853833
2,0,-0.091515,1.395827,-2.142008,-1.868485,-1.927902,-2.739654,-1.362059,-3.020756,1.786407,-1.390254
3,1,0.408198,-2.222789,4.955917,1.447182,2.875639,4.353920,2.440398,-1.753928,-0.860575,1.467571
4,1,-2.476690,2.037121,0.601976,-2.112301,0.398657,1.617501,0.537919,-0.176931,0.091872,-0.679397


# GC

In [31]:
import gc 
gc.collect()

0